## Objective

- optimize the agents using crew ai
- also ask agents to code
- generate suggestions for improvements
- organize the data into tabular insights
- plot charts to visualize trends
- full report


In [14]:
# Warning control
import warnings
import os, yaml

warnings.filterwarnings("ignore")

from crewai import Agent, Task, Crew
from dotenv import load_dotenv

load_dotenv()

os.environ["OPENAI_MODEL_NAME"] = os.getenv("OPENAI_MODEL_NAME")
os.environ["SERPER_API_KEY"] = os.getenv("SERPER_API_KEY")
os.environ["GOOGLE_GEMINI"] = os.getenv("GOOLE_GEMINI")

In [15]:
# Define file paths for YAML configurations
files = {"agents": "config/agents_perfOpti.yaml", "tasks": "config/tasks_perfOpti.yaml"}

# Load configurations from YAML files
configs = {}
for config_type, file_path in files.items():
    with open(file_path, "r") as file:
        configs[config_type] = yaml.safe_load(file)

# Assign loaded configurations to specific variables
agents_config = configs["agents"]
tasks_config = configs["tasks"]

## Reading tickets data


In [16]:
from crewai_tools import FileReadTool

csv_tool = FileReadTool(file_path="./support_tickets_data.csv")

In [17]:
import pandas as pd

df = pd.read_csv("./support_tickets_data.csv")
df.head()

,ticket_id,customer_id,issue_type,issue_description,priority,date_submitted,response_time_minutes,resolution_time_minutes,satisfaction_rating,customer_comments,agent_id,resolved
0,T0001,C0511,API Issue,I'm pleased with how my issue was handled. Tha...,High,2023-03-25,240,927,4,I'm pleased with how my issue was handled. Tha...,A004,True
1,T0002,C0729,Login Issue,Excellent service! The agent went above and be...,Low,2023-04-06,223,534,5,The problem still persists. Not resolved yet.,A004,False
2,T0003,C0439,Report Generation,Resolution was satisfactory but could be impro...,Low,2023-04-25,214,592,1,"The issue was escalated quickly, which was app...",A004,False
3,T0004,C0100,Data Import,Agent was very helpful and polite. Great service!,High,2023-02-20,110,864,2,The issue was resolved quickly. Very satisfied!,A003,True
4,T0005,C0258,Feature Request,"My issue was handled well, but follow-up could...",Medium,2023-01-18,151,193,2,The problem still persists. Not resolved yet.,A004,False


## agents, tasks and crew


In [18]:
# Creating Agents
suggestion_generation_agent = Agent(
    config=agents_config["suggestion_generation_agent"], tools=[csv_tool]
)

reporting_agent = Agent(config=agents_config["reporting_agent"], tools=[csv_tool])

chart_generation_agent = Agent(
    config=agents_config["chart_generation_agent"], allow_code_execution=False
)
## allow_code_execution:  will execute the code in a protected environment

# Creating Tasks
suggestion_generation = Task(
    config=tasks_config["suggestion_generation"], agent=suggestion_generation_agent
)

table_generation = Task(config=tasks_config["table_generation"], agent=reporting_agent)

chart_generation = Task(
    config=tasks_config["chart_generation"], agent=chart_generation_agent
)

final_report_assembly = Task(
    config=tasks_config["final_report_assembly"],
    agent=reporting_agent,
    context=[suggestion_generation, table_generation, chart_generation],
)


# Creating Crew
support_report_crew = Crew(
    agents=[suggestion_generation_agent, reporting_agent, chart_generation_agent],
    tasks=[
        suggestion_generation,
        table_generation,
        chart_generation,
        final_report_assembly,
    ],
    verbose=True,
)

## Testing our CREW


In [24]:
# support_report_crew.test(n_iterations=1, eval_llm="gpt-3.5-turbo")

# ## try to increase the number of iterations:
# ## since it will help us to get the avg score much better

## Training our CREW


In [21]:
training_data = {
    str(suggestion_generation_agent.id): {
        0: {
            "initial_output": "Use a bar chart to compare regions.",
            "improved_output": "Use a stacked bar chart to compare regions and highlight the differences over time.",
            "human_feedback": "The improved output offers a clearer visualization and includes temporal context, making it more insightful.",
        }
    }
}

In [23]:
# support_report_crew.train(n_iterations=1, filename="training.pkl", inputs=training_data)

## Running the crew


In [25]:
result = support_report_crew.kickoff()

╭──────────────────────────────────────────── Crew Execution Started ─────────────────────────────────────────────╮
│                                                                                                                 │
│  Crew Execution Started                                                                                         │
│  Name: crew                                                                                                     │
│  ID: 3143f336-35eb-4ed2-8e90-51325c41c60b                                                                       │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

🚀 Crew: crew
└── 📋 Task: 4868ce5f-bf09-493f-98ac-347730fdc401
       Status: Executing Task...

🚀 Crew: crew
└── 📋 Task: 4868ce5f-bf09-493f-98ac-347730fdc401
       Status: Executing Task...
    └── 🤖 Agent: Suggestion Engine
        
            Status: In Progress

# Agent: Suggestion Engine
## Task: Generate actionable suggestions for resolving each classified support ticket. The suggestions should be based on: - Issue Type: Tailor suggestions to the specific type of issue reported. - Historical Data: Use historical data such as resolution_time_minutes and
  satisfaction_rating to inform the suggestions.
- Customer Feedback: Incorporate insights from customer_comments to
  customize the suggestions further.

The goal is to provide clear, actionable steps that the support team can take to resolve each issue efficiently and effectively.



🤖 Agent: Suggestion Engine

    Status: In Progress

...

# Agent: Report Generator
## Task: Assemble the final report by integrating the tables and charts generated in the previous tasks. The report should include: - Issue Classification Results: Overview of the types of issues reported,
  their frequency, and priority levels, presented in tables and charts.
- Agent Performance: Insights into the performance of support agents, based
  on metrics like resolution time and customer satisfaction, presented in
  tables and charts.
- Customer Satisfaction: A summary of the customer satisfaction ratings and
  their trends over time, presented in tables and charts.
- Suggested Actions: A summary of the actionable suggestions generated for
  each category of issue.

The report should have tables and relevant charts side by side with an appropriate size.
The report should be formatted for easy consumption by stakeholders, providing valuable insights into the performance of the support system and areas for improvement.



# Agent: Report Generator
## F

🚀 Crew: crew
├── 📋 Task: 4868ce5f-bf09-493f-98ac-347730fdc401
│      Assigned to: Suggestion Engine
│   
│      Status: ✅ Completed
│   └── 🤖 Agent: Suggestion Engine
│       
│           Status: ✅ Completed
├── 📋 Task: 1d3ad3a9-cd1e-4418-9ca9-a599e08de001
│      Assigned to: Report Generator
│   
│      Status: ✅ Completed
│   └── 🤖 Agent: Report Generator
│       
│           Status: ✅ Completed
├── 📋 Task: 200ffb93-4a25-44b1-94cb-6fe5158e95dc
│      Assigned to: Chart Specialist
│   
│      Status: ✅ Completed
│   └── 🤖 Agent: Chart Specialist
│       
│           Status: ✅ Completed
└── 📋 Task: 57c81c33-4755-4252-91fc-d328a1dc4c5d
       Status: Executing Task...
    └── 🤖 Agent: Report Generator
        
            Status: ✅ Completed

🚀 Crew: crew
├── 📋 Task: 4868ce5f-bf09-493f-98ac-347730fdc401
│      Assigned to: Suggestion Engine
│   
│      Status: ✅ Completed
│   └── 🤖 Agent: Suggestion Engine
│       
│           Status: ✅ Completed
├── 📋 Task: 1d3ad3a9-cd1e-4418-9ca9-a599e08de001
│      Assigned to: Report Generator
│   
│      Status: ✅ Completed
│   └── 🤖 Agent: Report Generator
│       
│           Status: ✅ Completed
├── 📋 Task: 200ffb93-4a25-44b1-94cb-6fe5158e95dc
│      Assigned to: Chart Specialist
│   
│      Status: ✅ Completed
│   └── 🤖 Agent: Chart Specialist
│       
│           Status: ✅ Completed
└── 📋 Task: 57c81c33-4755-4252-91fc-d328a1dc4c5d
       Assigned to: Report Generator
    
       Status: ✅ Completed
    └── 🤖 Agent: Report Generator
        
            Status: ✅ Completed

╭──────────────────────────────────────────────── Task Completion ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Task Completed                                                                                                 │
│  Name: 57c81c33-4755-4252-91fc-d328a1dc4c5d                                                                     │
│  Agent: Report Generator                                                                                        │
│                                                                                                                 │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭──────────────────────────────────────────────── Crew Completion ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Crew Execution Completed                                                                                       │
│  Name: crew                                                                                                     │
│  ID: 3143f336-35eb-4ed2-8e90-51325c41c60b                                                                       │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

In [26]:
from IPython.display import display, Markdown

display(Markdown(result.raw))

**Final Support System Performance Report**

---

### Issue Classification Results

#### Table: Issue Overview
| Issue Type        | Count | High Priority | Low Priority | Critical Priority | Average Resolution Time (min) |
|-------------------|-------|---------------|--------------|-------------------|-------------------------------|
| API Issue         | 5     | 1             | 0            | 2                 | 524.2                         |
| Login Issue       | 5     | 2             | 2            | 1                 | 629.0                         |
| Report Generation | 5     | 2             | 3            | 0                 | 625.2                         |
| Data Import       | 10    | 4             | 3            | 0                 | 624.5                         |
| Feature Request   | 5     | 3             | 0            | 1                 | 550.4                         |
| UI Bug            | 5     | 1             | 3            | 1                 | 608.0                         |
| Billing Issue     | 10    | 4             | 1            | 2                 | 734.0                         |

**Charts:**
- **Issue Distribution Chart**: Pie Chart showing the distribution of issue types.
- **Priority Levels Chart**: Stacked Bar Chart displaying High, Low, and Critical priority issues for each type.

### Agent Performance

#### Table: Performance Metrics
| Agent ID | Total Resolved | Total Not Resolved | Average Satisfaction Rating | Average Resolution Time (min) |
|----------|----------------|--------------------|-----------------------------|-------------------------------|
| A001     | 4              | 6                  | 2.9                         | 602.3                         |
| A002     | 4              | 4                  | 3.3                         | 792.0                         |
| A003     | 5              | 3                  | 3.1                         | 654.3                         |
| A004     | 7              | 9                  | 3.3                         | 565.8                         |
| A005     | 2              | 5                  | 2.9                         | 845.3                         |

**Chart:** 
- **Agent Performance Chart**: Scatter Plot correlating Average Resolution Time and Average Satisfaction Rating for each agent.

### Customer Satisfaction

#### Table: Monthly Satisfaction Ratings
| Month   | Average Rating |
|---------|----------------|
| January | 2.3            |
| February| 2.9            |
| March   | 3.1            |
| April   | 3.7            |
| May     | 2.8            |
| June    | 3.0            |

**Chart:**
- **Customer Satisfaction Chart**: Bar Chart visualizing the trend in customer satisfaction ratings over the months.

### Suggested Actions

**API Issue:**
1. Monitor API performance continuously.
2. Provide immediate technical support by escalating to technical teams.
3. Engage with customers to update them regularly on the progress.

**Login Issue:**
1. Verify user account status and login credentials.
2. Reset passwords promptly when necessary.
3. Enhance customer communication about security updates and access issues.

*(Continue in this manner for each issue type.)*

---

**Concluding Remarks:**
This report provides a comprehensive overview of the current state of the support system, highlighting key areas for improvement and offering actionable suggestions to enhance performance and customer satisfaction. This document should serve as a roadmap for strategic decisions in customer support operations.